$ \huge{Proyecto \ de \ Deep \ Learning: Implementaci \acute{o} n \ Albert} $


Primero que todo revisamos que GPU nos han asignado

In [ ]:
!nvidia-smi

Tue Jul 13 16:21:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Clonamos los transformadores desde el github de Hugging Face


In [ ]:
!git clone https://github.com/huggingface/transformers \
&& cd transformers \
&& git checkout a3085020ed0d81d4903c50967687192e3101e770 

Cloning into 'transformers'...
remote: Enumerating objects: 78235, done.
remote: Counting objects: 100% (1276/1276), done.
remote: Compressing objects: 100% (776/776), done.
remote: Total 78235 (delta 769), reused 886 (delta 448), pack-reused 76959
Receiving objects: 100% (78235/78235), 61.09 MiB | 27.08 MiB/s, done.
Resolving deltas: 100% (55727/55727), done.
Note: checking out 'a3085020ed0d81d4903c50967687192e3101e770'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at a3085020e Added repetition penalty to PPLM example (#2436)


Instalaremos transformers y tensorboardX

In [ ]:
!pip install ./transformers
!pip install tensorboardX

Processing ./transformers
     |████████████████████████████████| 4.7MB 13.0MB/s 
     |████████████████████████████████| 133kB 33.7MB/s 
     |████████████████████████████████| 1.2MB 34.1MB/s 
     |████████████████████████████████| 901kB 37.2MB/s 
     |████████████████████████████████| 81kB 10.0MB/s 
     |████████████████████████████████| 7.7MB 37.4MB/s 
  Created wheel for transformers: filename=transformers-2.3.0-cp37-none-any.whl size=458567 sha256=6e556416be0e830d6f23596cf569e3ef3333ca883ee5a5a8cb6b4aa1e2fb1086
  Stored in directory: /tmp/pip-ephem-wheel-cache-noerjls2/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
ERROR: botocore 1.20.110 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 133kB 14.7MB/s 


Descargamos los datasets que queremos usar.

In [ ]:
!mkdir dataset \
&& cd dataset \
&& wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json \
&& wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2021-07-13 16:22:19--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   121MB/s    in 0.3s    

2021-07-13 16:22:21 (121 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2021-07-13 16:22:21--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

Entrenaremos 8 epocas.

In [ ]:
!export SQUAD_DIR=/content/dataset \
&& python transformers/examples/run_squad.py \
  --model_type albert \
  --model_name_or_path albert-base-v2 \
  --do_train \
  --do_eval \
  --do_lower_case \
  --train_file $SQUAD_DIR/train-v2.0.json \
  --predict_file $SQUAD_DIR/dev-v2.0.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 8.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir /content/model_output \
  --save_steps 1000 \
  --threads 4 \
  --version_2_with_negative 

Se han truncado las últimas 5000 líneas del flujo de salida.
Iteration:  56% 6181/11017 [49:46<38:56,  2.07it/s]
Iteration:  56% 6182/11017 [49:47<38:55,  2.07it/s]
Iteration:  56% 6183/11017 [49:47<38:56,  2.07it/s]
Iteration:  56% 6184/11017 [49:48<38:57,  2.07it/s]
Iteration:  56% 6185/11017 [49:48<38:56,  2.07it/s]
Iteration:  56% 6186/11017 [49:48<38:56,  2.07it/s]
Iteration:  56% 6187/11017 [49:49<38:55,  2.07it/s]
Iteration:  56% 6188/11017 [49:49<38:52,  2.07it/s]
Iteration:  56% 6189/11017 [49:50<38:51,  2.07it/s]
Iteration:  56% 6190/11017 [49:50<38:50,  2.07it/s]
Iteration:  56% 6191/11017 [49:51<38:53,  2.07it/s]
Iteration:  56% 6192/11017 [49:51<38:51,  2.07it/s]
Iteration:  56% 6193/11017 [49:52<38:49,  2.07it/s]
Iteration:  56% 6194/11017 [49:52<38:50,  2.07it/s]
Iteration:  56% 6195/11017 [49:53<38:51,  2.07it/s]
Iteration:  56% 6196/11017 [49:53<38:51,  2.07it/s]
Iteration:  56% 6197/11017 [49:54<38:49,  2.07it/s]
Iteration:  56% 6198/11017 [49:54<38:47,  2.07it/s]
Ite

In [ ]:
import os
import torch
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import (
    AlbertConfig,
    AlbertForQuestionAnswering,
    AlbertTokenizer,
    squad_convert_examples_to_features
)

from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample

from transformers.data.metrics.squad_metrics import compute_predictions_logits

# READER NOTE: Set this flag to use own model, or use pretrained model in the Hugging Face repository
use_own_model = True

if use_own_model:
  model_name_or_path = "/content/model_output"
else:
  model_name_or_path = "ktrapeznikov/albert-xlarge-v2-squad-v2"

output_dir = ""

# Config
n_best_size = 1
max_answer_length = 30
do_lower_case = True
null_score_diff_threshold = 0.0

def to_list(tensor):
    return tensor.detach().cpu().tolist()

# Setup model
config_class, model_class, tokenizer_class = (
    AlbertConfig, AlbertForQuestionAnswering, AlbertTokenizer)
config = config_class.from_pretrained(model_name_or_path)
tokenizer = tokenizer_class.from_pretrained(
    model_name_or_path, do_lower_case=True)
model = model_class.from_pretrained(model_name_or_path, config=config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

processor = SquadV2Processor()

def run_prediction(question_texts, context_text):
    """Setup function to compute predictions"""
    examples = []

    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            is_impossible=False,
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=384,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    output_prediction_file = "predictions.json"
    output_nbest_file = "nbest_predictions.json"
    output_null_log_odds_file = "null_predictions.json"

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,  # verbose_logging
        True,  # version_2_with_negative
        null_score_diff_threshold,
        tokenizer,
    )

    return predictions

Testeamos nuestro modelo.

In [ ]:
context = "New Zealand (Māori: Aotearoa) is a sovereign island country in the southwestern Pacific Ocean. It has a total land area of 268,000 square kilometres (103,500 sq mi), and a population of 4.9 million. New Zealand's capital city is Wellington, and its most populous city is Auckland."
questions = ["How many people live in New Zealand?", 
             "What's the largest city?"]

# Run method
predictions = run_prediction(questions, context)

print("\n")
print("\n")

# Print results

for i in range(len(questions)):
  question = questions[i]
  answer = predictions[str(i)]
  print("-----------------------")
  print("Question: ", question)
  print("Answer: ", answer)
  print("-----------------------")

add example index and unique id: 100%|██████████| 2/2 [00:00<00:00, 2490.68it/s]





-----------------------
Question:  How many people live in New Zealand?
Answer:  4.9 million.
-----------------------
-----------------------
Question:  What's the largest city?
Answer:  Auckland
-----------------------


In [ ]:
context = "Cristiano Ronaldo dos Santos Aveiro GOIH ComM (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaɫdu]; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for Serie A club Juventus and captains the Portugal national team. Often considered the best player in the world and widely regarded as one of the greatest players of all time, Ronaldo has won five Ballons d'Or and four European Golden Shoes, both of which are records for a European player. He has won 32 major trophies in his career, including seven league titles, five UEFA Champions Leagues, one UEFA European Championship, and one UEFA Nations League title. Ronaldo holds the records for the most international goals (109, tied with Ali Daei), the most goals (134) and assists (42) in the Champions League, and the most goals in the European Championship (14). He is one of the few recorded players to have made over 1,100 professional career appearances and has scored over 780 official senior career goals for club and country. In a 2012 interview, Ronaldo commented on the rivalry, saying: I think we push each other sometimes in the competition, this is why the competition is so high. Alex Ferguson, Ronaldo's manager during his time at Manchester United, opined: I don't think the rivalry against each other bothers them. I think they have their own personal pride in terms of wanting to be the best. Messi himself denied any rivalry, saying that it was only the media, the press, who wants us to be at loggerheads but I've never fought with Cristiano. Responding to the claims that he and Messi do not get on well on a personal level, Ronaldo commented: We don't have a relationship outside the world of football, just as we don't with a lot of other players. Ronaldo added that in years to come he hopes they can laugh about it together, stating: We have to look on this rivalry with a positive spirit, because its a good thing. Representing archrivals Barcelona and Real Madrid, the two players faced each other at least twice every season in the worlds biggest club game, El Clásico, which is among the worlds most viewed annual sporting events."

questions = ["Who is the best player of the world?", 
             "In which club plays Cristiano Ronaldo?", 
             "How many goals scored Cristiano Ronaldo in all his carrer?",
             "Who was the Cristiano Ronaldo Manager at Manchester United?",
             "How many golden boots has won Cristiano Ronaldo?",
             "Who is Barack Obama?"]

# Run method
predictions = run_prediction(questions, context)

print("\n")
print("\n")

# Print results

for i in range(len(questions)):
  question = questions[i]
  answer = predictions[str(i)]
  print("-----------------------")
  print("Question: ", question)
  print("Answer: ", answer)
  print("-----------------------")

add example index and unique id: 100%|██████████| 6/6 [00:00<00:00, 25575.02it/s]






-----------------------
Question:  Who is the best player of the world?
Answer:  Cristiano Ronaldo
-----------------------
-----------------------
Question:  In which club plays Cristiano Ronaldo?
Answer:  Juventus
-----------------------
-----------------------
Question:  How many goals scored Cristiano Ronaldo in all his carrer?
Answer:  780
-----------------------
-----------------------
Question:  Who was the Cristiano Ronaldo Manager at Manchester United?
Answer:  Alex Ferguson
-----------------------
-----------------------
Question:  How many golden boots has won Cristiano Ronaldo?
Answer:  four
-----------------------
-----------------------
Question:  Who is Barack Obama?
Answer:  
-----------------------
